<a href="https://colab.research.google.com/github/caro28/stinky/blob/master/oil_vessel_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summary

*   This file loads and merges two xlsx spreadsheets, which include oil vessel arrival data into Portland for 2020 and 2021.
* In addition, it accounts for the possibility that future data will be received in new xlsx spreadsheets. Any new spreadsheets should be saved in the [data_vessels folder](https://github.com/ds5110/stinky/tree/master/vessels_data).
*   Key columns are: Arrival date, Terminal (for final delivery), Product Type, Product Amount (Incoming and Outgoing)
*   Possible key columns with empty or mostly empty data: Arrival time.
*   Separate columns were added to look at dates more granularly.

# Instructions for updating:
First, in Excel: user uploading data from Excel should do the following first:

1. data should be on one sheet only. if data is pulled from other sheets in Excel file, user should copy/paste (values only) data in place, then delete all other sheets.
2. delete empty rows above column names.
3. column names should be in one row only. Unmerge any merged rows that contain column names. Delete all unmerged rows that do not contain the main column name.
4. ensure column names match the order of the sequence below. If not, then move columns so that they remain in this order. Do not rename columns to match names below; this notebook will rename them.

> 'ARRIVAL DATE', 'ARRIVAL TIME (if avail)', 'TERMINAL', 'SHIP NAME', 'LAST PORT OF CALL (if avail)', 'PRODUCT TYPE - OIL', 'PRODUCT AMOUNT - OIL, Incoming (Multiply metric tons by 7.5 for approx bbls)', 'PRODUCT AMOUNT - OIL, Outgoing', 'RUNNING TOTAL (Oil)', 'PRODUCT TYPE - NON-OIL',	'PRODUCT AMOUNT - NON OIL, Incoming', 'PRODUCT AMOUNT - NON OIL, Outgoing', 'Running Total (Specify units) (Non-Oil)'

5. check that all dates have been entered correctly (e.g. original 2021 Excel file contained date "2/13/20221" - errors such as this should be fixed in Excel to 2/13/2021).
6. Delete any empty rows at the end of the sheet.
7.   save as csv.

Second, on GitHub:

1. If receive new data in a new spreadsheet: first, go through steps above in Excel.
2. Upload new spreadsheet (csv) to [data_vessels folder](https://github.com/ds5110/stinky/tree/master/vessels_data). Filename does not matter but must end in '.csv'.
3. Open this notebook in Google colab. 
4. Runtime --> Run all: this will download the updated df_vessels table.
5. Save as df_vessels.csv and upload to [data folder](https://github.com/ds5110/stinky/tree/master/data). (Do not need to delete old df_vessels.csv file; GitHub will automatically replace with new file.)


In [ ]:
# TODO: delete this cell if repo is made public

# user: ADD GITHUB USERNAME AND PASSWORD HERE, inside quotation marks
user_name = ''
password = ''
repo = 'ds5110/stinky'

In [ ]:
pip install PyGithub

     |████████████████████████████████| 291 kB 5.2 MB/s 
     |████████████████████████████████| 961 kB 37.3 MB/s 


In [ ]:
# TODO: temporary access through Caroline's GitHub - delete
user_name = "caro28"
password = "phid-knod7gack5TURT"

# connect to vessels_data folder on GitHub
def connect_to_git(user_name, password, repo):
  from github import Github
  # First create a Github instance:

  g = Github(user_name, password)

  repo = g.get_repo(repo)
  return repo.get_contents('vessels_data')

file_contents = connect_to_git(user_name, password, repo)

In [ ]:
import pandas as pd
import string

# get files in vessels_data and merge into one dataframe, df_vessels
def get_data(file_contents):
  collector = []
  for file in file_contents:
    collector.append(pd.read_csv(file.download_url, header=None, skiprows=1))
  return pd.concat(collector, ignore_index=True)

df_vessels = get_data(file_contents)

In [ ]:
# rename columns
col_names = ['ARRIVAL DATE', 'ARRIVAL TIME (if avail)', 'TERMINAL', 
             'SHIP NAME', 'LAST PORT OF CALL (if avail)', 'PRODUCT TYPE - OIL',
             'PRODUCT AMOUNT - OIL, Incoming (Multiply metric tons by 7.5 for approx bbls)',
             'PRODUCT AMOUNT - OIL, Outgoing', 'RUNNING TOTAL (Oil)',
             'PRODUCT TYPE - NON-OIL', 'PRODUCT AMOUNT - NON OIL, Incoming',
             'PRODUCT AMOUNT - NON OIL, Outgoing', 'Running Total (Specify units) (Non-Oil)']

df_vessels.columns = col_names

In [ ]:
def separate_date_cols(df):
  '''
  Function: separate_date_cols()
    Converts 'Arrival Date' column to datetime object and creates new, separate 
    columns for year, month, day, and month_name
  Parameters:
    df: (pandas.DataFrame object) input dataframe (df_vessels)
  Returns:
    None
  '''
  df['ARRIVAL DATE']=pd.to_datetime(df['ARRIVAL DATE'])
  df['Year']=df['ARRIVAL DATE'].dt.year
  df['Month']=df['ARRIVAL DATE'].dt.month
  df['Day']=df['ARRIVAL DATE'].dt.day
  df['Month_name'] = pd.to_datetime(df['Month'], format='%m').dt.month_name().str.slice(stop=3)

def strip_char(df):
  '''
  Function: strip_char(df)
    Removes digits and special characters from 'Product type - oil' column
  Parameters:
    df: (pandas.DataFrame object) input dataframe (df_vessels)
  Returns:
    None
  '''
  #df['PRODUCT TYPE - OIL'] = df['PRODUCT TYPE - OIL'].str.lstrip(string.digits) # Aishwarya will edit this line to keep product codes but edit formatting of 01, 02, 06 to match all other codes
  df['PRODUCT TYPE - OIL'] = df['PRODUCT TYPE - OIL'].replace(['#','-'],'',regex=True)

In [ ]:
# create separate columns for year/month/day
separate_date_cols(df_vessels)

# strip characters from 'Product Type - Oil"
strip_char(df_vessels)

In [ ]:
# sort chronologically
df_vessels=df_vessels.sort_values(by='ARRIVAL DATE')

# reset index
df_vessels=df_vessels.reset_index(drop=True)

In [ ]:
def convert_float(col):
  '''
  Function: convert_float()
  Parameters:
    col: (pandas.Series object) column whose values will be converted to float
  Returns:
    (list) column values in a list, converted to float
  '''
  return [float(str(x).replace(',', '')) for x in col]

def replace_nan_0(col):
  '''
  Function: replace_nan()
    Replaces any NaN values in input with 0
  Parameters:
    col: (pandas.Series object) column whose NaN values will be replaced with 0
  Returns:
    None
  '''
  col.fillna(0, inplace=True)

In [ ]:
# convert numerical data columns to float
df_vessels['PRODUCT AMOUNT - OIL, Incoming (Multiply metric tons by 7.5 for approx bbls)'] = convert_float(df_vessels['PRODUCT AMOUNT - OIL, Incoming (Multiply metric tons by 7.5 for approx bbls)'])
df_vessels['PRODUCT AMOUNT - OIL, Outgoing'] = convert_float(df_vessels['PRODUCT AMOUNT - OIL, Outgoing'])
df_vessels['Running Total (Specify units) (Non-Oil)'] = convert_float(df_vessels['Running Total (Specify units) (Non-Oil)'])

# replace NaN with 0
replace_nan_0(df_vessels['PRODUCT AMOUNT - OIL, Outgoing'])
replace_nan_0(df_vessels['PRODUCT AMOUNT - OIL, Incoming (Multiply metric tons by 7.5 for approx bbls)'])
replace_nan_0(df_vessels['PRODUCT AMOUNT - NON OIL, Incoming'])
replace_nan_0(df_vessels['PRODUCT AMOUNT - NON OIL, Outgoing'])

In [ ]:
def correct_runningtotal(incoming, outgoing):
  '''
  Function: correct_runningtotal()
    Re-calculates values in 'Running Total' columns (net cumulative sums), due 
    to errors in original values in Excel files and to maintain the cumulative
    sum across merged spreadsheets.
  Paramaters:
    incoming: (pandas.Series object) column with values to be added to 
      cumulative sum
    outgoing: (pandas.Series object) column with values to be subtracted from 
      cumulative sum
  Returns:
    running_total: (list) new values for 'Running total' columns
  '''
  running_total = []
  net = incoming[0]
  running_total.append(net)
  for i in range(1, len(df_vessels)):
    net += incoming[i]
    net -= outgoing[i]
    running_total.append(net)
  return running_total

In [ ]:
# correct 'Running total (Oil)'
incoming = df_vessels['PRODUCT AMOUNT - OIL, Incoming (Multiply metric tons by 7.5 for approx bbls)']
outgoing = df_vessels['PRODUCT AMOUNT - OIL, Outgoing']
df_vessels['RUNNING TOTAL (Oil)'] = correct_runningtotal(incoming, outgoing)

# correct 'Running total (Non-oil)'
incoming = df_vessels['PRODUCT AMOUNT - NON OIL, Incoming']
outgoing = df_vessels['PRODUCT AMOUNT - NON OIL, Outgoing']
df_vessels['Running Total (Specify units) (Non-Oil)'] = correct_runningtotal(incoming, outgoing)

In [ ]:
df_vessels.isna().sum()

ARRIVAL DATE                                                                      0
ARRIVAL TIME (if avail)                                                         770
TERMINAL                                                                          0
SHIP NAME                                                                         0
LAST PORT OF CALL (if avail)                                                     67
PRODUCT TYPE - OIL                                                                0
PRODUCT AMOUNT - OIL, Incoming (Multiply metric tons by 7.5 for approx bbls)      0
PRODUCT AMOUNT - OIL, Outgoing                                                    0
RUNNING TOTAL (Oil)                                                               0
PRODUCT TYPE - NON-OIL                                                          770
PRODUCT AMOUNT - NON OIL, Incoming                                                0
PRODUCT AMOUNT - NON OIL, Outgoing                                          

In [ ]:
# download tidied df to user's computer (local files)
from google.colab import files
df_vessels.to_csv('df_vessels.csv', index=False)
files.download('df_vessels.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>